In [1]:
import pandas as pd
import numpy as np

In [4]:
products=pd.read_csv("amazon_baby.csv")
products = products.fillna({'review':''})

In [5]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [6]:
def remove_punctuation(text):
    import string #still good for vairables like puntcuation 
    import re
    
    punctuation=string.punctuation #list of punctuation simbols
    clean=re.sub(r'[{}]'.format(punctuation),'',text) #make pattern with symbols and replace them
    
    return clean

In [7]:
products["review clean"]=products["review"].apply(remove_punctuation)


In [41]:
# ignore reviews with ratin=3 --> neutral sentiment

products=products[products["rating"] !=3].copy()
len(products)

166752

In [9]:
#reviews with rating 4+ are positive, revs with rating 2- are negative
products["sentiment"]=products["rating"].apply(lambda rating: +1 if rating >3 else -1)

In [20]:
#split data

test_index=pd.read_json("module-2-assignment-test-idx.json", typ="series")
train_index=pd.read_json("module-2-assignment-train-idx.json", typ="series")


In [11]:
test=products.iloc[test_index]

train=products.iloc[train_index]
#could screw mw over, since index 

In [23]:
# make vectorizer

from sklearn.feature_extraction.text import CountVectorizer

vectorizer=CountVectorizer(token_pattern=r"\b\w+\b") #keeps single letter words

#learn vocabulary from data and assign columns to words
#then convert data into sparse matrix
train_matrix=vectorizer.fit_transform(train["review clean"])
test_matrix=vectorizer.transform(test["review clean"])

In [24]:
# train clf with log regresson
X_train=train_matrix.copy()
y_train=train["sentiment"].copy()

from sklearn.linear_model import LogisticRegression
sentiment_model=LogisticRegression().fit(X_train, y_train)

In [25]:
sentiment_model.coef_.shape

(1, 121706)

# Q1 How many weights are greater than or equal to 0?

Q1=86973

In [44]:
Q1=(sentiment_model.coef_ >=0).sum()
neg=(sentiment_model.coef_ <0).sum()

In [46]:
Q1, neg

(86973, 34733)

# Q2 Of the three data points in sample_test_data, which one has the lowest probability of being classified as a positive review?

Q2=third

In [33]:
sample_test_data=test[10:13]

In [35]:
sample_test_matrix=vectorizer.transform(sample_test_data["review clean"])
scores=sentiment_model.decision_function(sample_test_matrix)
print(scores)

[  5.60115395  -3.16926451 -10.42289435]


In [47]:
def class_prediction(scores):
    
    preds=[]
    for score in scores:
        if score >0:
            pred=1
        else:
            pred=-1
        preds.append(pred)
    return preds

In [48]:
class_prediction(scores)

[1, -1, -1]

In [49]:
#check
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1], dtype=int64)

In [83]:
def calc_prob(scores):
    """ Calculate the probability predictions from the scores.
    """
    import numpy as np
    #scores=scores.reshape([1,3])
    prob_preds=[]
    for score in scores:
        prob_pred=1/( 1+np.exp(-scores) )
        prob_preds.append(prob_pred)
    return prob_preds[1]

calc_prob(scores)

array([  9.96319993e-01,   4.03388779e-02,   2.97427814e-05])

In [84]:
#check
probs=sentiment_model.predict_proba(sample_test_matrix)[:,1]
probs

array([  9.96319993e-01,   4.03388779e-02,   2.97427814e-05])

# Q3 top 20 reviews

Q3= Britax decathlon.....

In [107]:
test_matrix=vectorizer.transform(test["review clean"])
test["predicted_proba"]=sentiment_model.predict_proba(test_matrix)[:,1]

In [114]:
top20=test.sort_values(by="predicted_proba", ascending=False)[:21]

In [115]:
top20

,name,review,rating,review clean,sentiment,predicted_proba
52631,Evenflo X Sport Plus Convenience Stroller - Ch...,After seeing this in Parent's Magazine and rea...,5,After seeing this in Parents Magazine and read...,1,1.0
80155,"Simple Wishes Hands-Free Breastpump Bra, Pink,...","I just tried this hands free breastpump bra, a...",5,I just tried this hands free breastpump bra an...,1,1.0
50315,"P'Kolino Silly Soft Seating in Tias, Green",I've purchased both the P'Kolino Little Reader...,4,Ive purchased both the PKolino Little Reader C...,1,1.0
87017,Baby Einstein Around The World Discovery Center,I am so HAPPY I brought this item for my 7 mon...,5,I am so HAPPY I brought this item for my 7 mon...,1,1.0
66059,"Evenflo 6 Pack Classic Glass Bottle, 4-Ounce",It's always fun to write a review on those pro...,5,Its always fun to write a review on those prod...,1,1.0
168081,Buttons Cloth Diaper Cover - One Size - 8 Colo...,"We are big Best Bottoms fans here, but I wante...",4,We are big Best Bottoms fans here but I wanted...,1,1.0
97325,Freemie Hands-Free Concealable Breast Pump Col...,I absolutely love this product. I work as a C...,5,I absolutely love this product I work as a Cu...,1,1.0
140816,"Diono RadianRXT Convertible Car Seat, Plum",I bought this seat for my tall (38in) and thin...,5,I bought this seat for my tall 38in and thin 2...,1,1.0
168697,Graco FastAction Fold Jogger Click Connect Str...,Graco's FastAction Jogging Stroller definitely...,5,Gracos FastAction Jogging Stroller definitely ...,1,1.0
22586,"Britax Decathlon Convertible Car Seat, Tiffany",I researched a few different seats to put in o...,4,I researched a few different seats to put in o...,1,1.0


# Q4 looser20 (most neg revs)

Q4= the first year..., Peg-perego, safety 1st high def monitor

In [118]:
looser=test.sort_values(by="predicted_proba")[:21]
looser

,name,review,rating,review clean,sentiment,predicted_proba
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,-1,8.348850e-16
120209,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,-1,1.591034e-15
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,-1,8.101970e-14
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,-1,9.767978e-14
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,-1,1.946115e-13
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,-1,3.258344e-13
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,-1,3.296419e-11
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,-1,3.309140e-11
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,-1,9.533157e-11
10677,Philips AVENT Newborn Starter Set,"It's 3am in the morning and needless to say, t...",1,Its 3am in the morning and needless to say thi...,-1,9.621635e-11


# Q5 accuracy on test data

Q5=0.93

In [120]:
test_matrix=vectorizer.transform(test["review clean"])
y_test=test["sentiment"]
print(sentiment_model.score(test_matrix,y_test))
#answer=0.93

0.932235421166


# second clf, less words

In [122]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [124]:
sub_vectorizer=CountVectorizer(vocabulary=significant_words)
train_sub_matrix=sub_vectorizer.fit_transform(train["review clean"])
test_sub_matrix=sub_vectorizer.transform(test["review clean"])

In [162]:
# train clf with log regresson

X_train_sub=train_sub_matrix.copy()
y_train_sub=train["sentiment"].copy()

from sklearn.linear_model import LogisticRegression
sentiment_model_sub=LogisticRegression().fit(X_train_sub, y_train_sub)

In [152]:
intercept=sentiment_model_sub.intercept_

In [155]:
coef=sentiment_model_sub.coef_.reshape(1,20)

In [149]:
coef_table_sub=pd.DataFrame({"word":significant_words, "coeff":coef.flatten()})
coef_table_sub

,coeff,word
0,1.363688,love
1,0.943998,great
2,1.192537,easy
3,0.085509,old
4,0.520183,little
5,1.509810,perfect
6,1.673118,loves
7,0.503759,well
8,0.190906,able
9,0.058854,car


# Q7 How many of the 20 coefficients (corresponding to the 20 significant_words and excluding the intercept term) are positive for the simple_model?

Q7=10

In [156]:
q7=(coef_table_sub >0).sum()
q7

coeff    10
word     20
dtype: int64

q8=yes

# Q8 Are the positive words in the simple_model also positive words in the sentiment_model?

Q8=yes

# Q9 Which model (sentiment_model or simple_model) has higher accuracy on the TRAINING set?


In [160]:
#1st model on train 
print(sentiment_model.score(train_matrix,y_train))

0.96849703184


In [163]:
#simple model on train 

print(sentiment_model_sub.score(X_train_sub, y_train_sub))

0.866822570007


Q9 which models has highest score on train?

Q10=which models has highest score on test?

q9=sentiment_model
q10=sentiment

# Q10 Which model (sentiment_model or simple_model) has higher accuracy on the TEST set?

In [158]:
#1st model on test
test_matrix=vectorizer.transform(test["review clean"])
y_test=test["sentiment"]
print(sentiment_model.score(test_matrix,y_test))

0.932235421166


In [165]:
#simple model

test_sub_matrix=sub_vectorizer.transform(test["review clean"])
y_test_sub=test["sentiment"]
print(sentiment_model_sub.score(test_sub_matrix,y_test_sub))

0.869360451164


# Q11 is the sentiment classifier better than dummy clf?

q11=0.84

In [168]:
from sklearn.dummy import DummyClassifier
#neg class 0 is most frequent
dummy_majority=DummyClassifier(strategy= "most_frequent").fit(test_matrix,y_test)
#therefore the dummy "most_freq" class always predicts class 0

y_dummy_predictions=dummy_majority.predict(test_matrix)
y_dummy_predictions
dummy_majority.score(test_matrix,y_test)

0.84278257739380846